## Exercise 3 ~ Interactive Password-based Authentication

In [1]:
import socket
import binascii
import random
import hashlib
import base64
from Crypto.Cipher import AES
import utils
# Get variable from parameters file
params = utils.get_parameters()

In [2]:
Q3_id = params.Q3_id

In [3]:
print Q3_id

Mj8X8EEctVs0s3


In [4]:
# TEST ANSWER
Q3_w = "lT4%&fFiP[fHSB"

In [5]:
server = None

def send_message(message):
    global server
    server_name = "lasecpc25.epfl.ch"
    port = "7777"
    if server == None:
        server = socket.create_connection((server_name, int(port)))
    server.send(bytes(message + "\n"))
    response=""
    while True: #data might come in several packets, need to wait for all of it
        data = server.recv(1024)
        data = data.decode("ASCII")
        if len(data) == 0:
            break
        if data[-1] == '\n': 
            response += data[:-1]  
            break
        response += data
    return response

def close_conn():
    global server
    if server != None:
        server.close()
        server = None

In [6]:
def base64_to_bin(m, l=128):
    return hex_to_bin(binascii.b2a_hex(binascii.a2b_base64(m)), l)

def hex_to_bin(h, l=128):
    return bin(int(h, 16))[2:].zfill(l)

def bin_to_dat(b, l=32):
    h = '%x' % int(b, 2)
    return binascii.unhexlify(h.zfill(l))

def bin_to_base64(b):
    return base64.b64encode(bin_to_dat(b))
    
def xor(a, b):
    c = ""
    for i in range(len(a)):
        if a[i] == b[i]:
            c += "0"
        else:
            c += "1"
    return c

In [7]:
def exchange_part_1(i):
    m_1 = ' ' * 128
    while m_1[i] != '0':
        close_conn()
        m_1_b64 = send_message(Q3_id)
        m_1 = base64_to_bin(m_1_b64)
    return m_1
    
def create_m_0(i, bit, m_1):
    k = None
    m_0 = ' ' * 128
    while k != i or m_0[i] != bit:
        m_0 = ''.join([str(random.randrange(2)) for _ in range(128)])
        to_hash = Q3_id + bin_to_dat(m_0) + bin_to_dat(m_1)
        c = hex_to_bin(hashlib.sha512(to_hash).hexdigest(), 512)[:256]
        k = int(c[0:8], 2) % 128
    return m_0


def exchange_part_2(m_0):
    m_0_b64 = bin_to_base64(m_0)
    ch_0 = send_message(m_0_b64)
    ch_1 = send_message(ch_0)
    return ch_0, ch_1

In [8]:
s_0 = ""
s_1 = ""

i = 0
while i < 128:
    # Initiate the connection and get m_1
    m_1 = exchange_part_1(i)
    # Generate the m_0 needed to get a good c
    m_0 = create_m_0(i, '0', m_1)
    ch_0, ch_1 = exchange_part_2(m_0)
    if ch_1 != 'abort':
        s_0 += ch_0
        s_1 += ch_0
    else:
        m_1 = exchange_part_1(i)
        m_0 = create_m_0(i, '1', m_1)
        ch_0_2, ch_1_2 = exchange_part_2(m_0)
        s_0 += xor('1', ch_0_2)
        s_1 += ch_0_2
    i += 1

In [9]:
aes_0 = AES.new(bin_to_dat('0' * 128))
aes_1 = AES.new(bin_to_dat('1' * 128))

print aes_0.decrypt(bin_to_dat(s_0))
print aes_1.decrypt(bin_to_dat(s_1))

6Nqqagj:$^$tjx  
6Nqqagj:$^$tjx  
